In [ ]:
!pip install pandas

In [1]:
import os
from fnmatch import fnmatch
import subprocess as sp
import pandas as pd

In [2]:
root = '../data/classical-domains'
pattern = "*.pddl"

In [3]:
dict_files = dict()
for path, subdirs, files in os.walk(root):
    top = {'domain':"",'problem':[]}
    for name in files:
        if fnmatch(name, pattern):
            if(path.replace(root,"") not in dict_files.keys()):
                dict_files[path.replace(root,"")] = top
            if(name.count("domain")>0):
                dict_files[path.replace(root,"")]['domain'] = os.path.join(path, name)
            else:
                temp = dict_files[path.replace(root,"")]['problem']
                temp.append(os.path.join(path, name))
                dict_files[path.replace(root,"")]['problem'] = temp

In [5]:
df = pd.DataFrame(data = None, columns=['domain_name','domain','problem','plan'])
temp_name = list()
temp_domain = list()
temp_problem = list()
temp_plan = list()

for domain in dict_files:
    domain_cont = open(dict_files[domain]['domain'], 'r').read()
    for prob_path in dict_files[domain]['problem']:
        problem_cont = open(prob_path,'r').read()
        run_comd = '../FastDownward/fast-downward.py ' + dict_files[domain]['domain']+' '+prob_path+' '+ '--search "astar(lmcut())"'
        output= sp.getoutput(run_comd)
        temp = output.split("Actual search time")[1].split("Plan length")[0].split("\n")
        temp = temp[1:]
        temp = temp[:-1]
        temp_name.append(domain)
        temp_domain.append(domain_cont)
        temp_problem.append(problem_cont)
        temp_plan.append("\n".join(temp))

df['domain_name'] = temp_name
df['domain'] = temp_domain
df['problem'] = temp_problem
df['plan'] = temp_plan

KeyboardInterrupt: 

In [ ]:
df.to_csv("classical-planning-data.csv")